In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')


In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [9]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [10]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [11]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [12]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [13]:
# 보통 5-폴드 교차 검증이나 10-폴드 교차 검증을 많이 사용한다.
# 사이킷런에는 cross_validate()라는 교차 검증 함수가 있다.

In [14]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01579046, 0.02419519, 0.02569222, 0.01788282, 0.02351713]), 'score_time': array([0.00141549, 0.0012939 , 0.00128603, 0.00855589, 0.00114417]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [15]:
# fit_time, score_time, test_score 처음 2개의 키는 각각 모델을 훈련한 시간, 검증하는 시간을 나타낸다. 각 키마다 5개의 숫자가 담겨 있다. 5-폴드 교차 검증이다.
# test_score는 이름은 test_score지만 검증 폴드의 점수이다. 

In [16]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [17]:
# cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않습니다.
# 훈련 세트는 train_test_split()으로 나누면 되지만, 교차 검증을 할 때 훈련 세트를 섞으려면 분할기를 지정해야한다.

In [18]:
# cross_validate() 함수는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용하고 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용한다.

In [19]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [20]:
# 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면 n_splits=10으로 하면 된다.

In [21]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


하이퍼 파라미터 튜닝

In [22]:
# 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터를 하이퍼파라미터라고 합니다.
# 모델마다 적게는 1~2개에서, 많게는 5~6개의 매개변수를 제공한다.
# GridSearchCV 클래스는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행한다.

In [23]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=1) 
# GridSearchCV의 cv 매개변수 기본값은 5입니다. 따라서 min_impurity_decrease 값마다 5-폴드 교차 검증을 수행합니다. 결국 5 * 5 = 25개의 모델을 훈련한다.
# n_jobs 매개변수에서 병렬 실행에 사용할 cpu 코어 수를 지정한다.

In [24]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [25]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [26]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [27]:
# 0.0001이 가장 좋은 값으로 선택 되었다.

In [28]:
# 다섯번의 교차 검증 점수 평균은 mean_test_score에 저장되어 있다.

In [29]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [30]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [ ]:
# page250에서 멈췄다.